# TensorFlow Estimator Objects

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('iris.csv')
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [4]:
df.columns = ['sepal_length','sepal_width','petal_length','petal_width','target']

In [5]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [8]:
df['target'] =df['target'].apply(int)
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [14]:
y = df['target']
X = df.drop('target',axis=1)

In [15]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [36]:
import tensorflow as tf

# Feature Columns

In [37]:
X.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

In [38]:
feat_cols = []
for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [39]:
feat_cols

[NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [40]:
input_fun = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,
                                   num_epochs=5,shuffle=True)
# Num_epochs: Processing all data at least once. Here it is capped at 5

In [41]:
classifier = tf.estimator.DNNClassifier(hidden_units=[10,20,10],n_classes=3,
                                       feature_columns=feat_cols)

W0727 22:12:28.332957 4611761600 estimator.py:1811] Using temporary folder as model directory: /var/folders/zl/26g5_l2n76q2ym00hmhpvb200000gn/T/tmpnomsl7gb


In [42]:
classifier.train(input_fn=input_fun,steps=50)

In [43]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [44]:
predictions = list(classifier.predict(input_fn=pred_fn))

In [45]:
predictions

[{'logits': array([-0.7372345 , -0.06837691,  0.32733867], dtype=float32),
  'probabilities': array([0.17089324, 0.3335846 , 0.4955222 ], dtype=float32),
  'class_ids': array([2]),
  'classes': array([b'2'], dtype=object),
  'all_class_ids': array([0, 1, 2], dtype=int32),
  'all_classes': array([b'0', b'1', b'2'], dtype=object)},
 {'logits': array([ 2.5753837, -1.1339725, -2.4130797], dtype=float32),
  'probabilities': array([0.96964103, 0.02374969, 0.0066092 ], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'all_class_ids': array([0, 1, 2], dtype=int32),
  'all_classes': array([b'0', b'1', b'2'], dtype=object)},
 {'logits': array([-1.5699612, -0.6264438,  0.8508969], dtype=float32),
  'probabilities': array([0.06745583, 0.17329416, 0.75925004], dtype=float32),
  'class_ids': array([2]),
  'classes': array([b'2'], dtype=object),
  'all_class_ids': array([0, 1, 2], dtype=int32),
  'all_classes': array([b'0', b'1', b'2'], dtype=object)},
 {'logits'

In [46]:
final_preds=[]
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [47]:
final_preds

[2,
 0,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 0]

In [48]:
from sklearn.metrics import classification_report,confusion_matrix

In [49]:
print(confusion_matrix(y_test,final_preds))
print('\n')
print(classification_report(y_test,final_preds))

[[19  0  0]
 [ 0  3 10]
 [ 0  0 13]]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.23      0.38        13
           2       0.57      1.00      0.72        13

    accuracy                           0.78        45
   macro avg       0.86      0.74      0.70        45
weighted avg       0.87      0.78      0.74        45

